In [1]:
import pandas as pd
from pathlib import Path

In [2]:
dataset_dir = Path(".") / "datasets"
qa_data_dir = dataset_dir / "generated_qa"
qa_data_files = list(qa_data_dir.glob("*.json"))
print(qa_data_files)

[PosixPath('datasets/generated_qa/result_kakaocorp.kanana_nano_2.json'), PosixPath('datasets/generated_qa/result_hyperclovax_seed_text_i.json'), PosixPath('datasets/generated_qa/result_exaone_3.5_2.4b_instruc.json'), PosixPath('datasets/generated_qa/result_google_gemma_3_4b_it_fr.json'), PosixPath('datasets/generated_qa/result_deepseek_deepseek_chat_.json'), PosixPath('datasets/generated_qa/result_gpt_4o_mini.json.json'), PosixPath('datasets/generated_qa/result_qwen_qwen3_4b_free.json.json')]


In [3]:
ragas_generated_original = pd.read_csv(dataset_dir / "translated_output.csv")
sampled_200_dataset = pd.read_csv(dataset_dir / "dataset_200.csv")
ragas_generated_original.shape, sampled_200_dataset.shape

print(ragas_generated_original.columns)
print(sampled_200_dataset.columns)

Index(['user_input', 'reference_outputs', 'reference', 'synthesizer_name'], dtype='object')
Index(['user_input', 'reference'], dtype='object')


In [4]:
# 200개 샘플에 해당하는 질문만 추출해서 새로운 데이터프레임 생성
ragas_200_reference = ragas_generated_original[
    ragas_generated_original["user_input"].isin(sampled_200_dataset["user_input"])
]
ragas_200_reference.reset_index(inplace=True, drop=True)
print(ragas_200_reference.shape)
print(ragas_200_reference.head(10))

(200, 4)
                                          user_input  \
0  안녕하세요, 벡스코가 부산 지하철 시스템에서 에스컬레이터 유지보수를 위한 전용 장소...   
1                      현재 구역 분류에 따른 수영 지역의 부산시 주차 요금   
2       동매역 1호선에서 지정된 만남의 장소 수에 대한 정보를 제공해 주실 수 있나요?   
3  부산시 교통 담당자로서 부산경찰청 범죄예방대응과의 순찰 업무에서 미남의 역할을 설명...   
4  부산 지하철의 시설 관리자として、홈페이지 시스템의 회원 등록 과정에서 휴대폰번호의 ...   
5  안녕하세요, 누군가 부산 지하철에 접이식 자전거를 주중 언제든지 가져갈 수 있는지 ...   
6            안녕하세요, 만덕이 3호선에 있는지 알려줄 수 있나요? 잘 모르겠어요.   
7  부산 지하철의 인프라 비용을 평가하는 재무 분석가로서, 4호선에 설치된 에스컬레이터...   
8  부산 지하철 시설 관리자입니다. 지하철역에서 비상 상황 발생 시 NIS국가정보원 연...   
9  공공 교통 기관을 위한 인프라 투자 평가를 담당하는 재무 분석가로서, 중앙역의 자동...   

                                   reference_outputs  \
0  ['총 42개소\n\n전용: 벡스코(시립미술관), 민락, 금련산, 문현, 냉정, 증...   
1  ['1급지: 시간제 주차 500원 (10분마다), 1일 주차 15,000원, 월 주...   
2  ['1호선 만남의 장소\n\n1호선 각 역의 만남의 장소 수량은 다음과 같습니다: ...   
3  ['현황\n\n부산경찰청 범죄예방대응과 기동순찰대는 경찰관 16명으로 구성되어 있으...   
4  ['약관은 홈페이지에 게시하거나 공지하여 효력을 발생합니다.\n\n변경된 약관은 공...   
5  ['메트로페이 충전 및 정기승차권 구매 가능하며, 도시철도만 이

In [5]:
# ragas_200_ref의 user_input의 데이터가 llm에 입력값과 일치하는지 검증

for col1, col2 in zip(
    sorted(sampled_200_dataset["user_input"]), sorted(ragas_200_reference["user_input"])
):
    if col1 != col2:
        print(col1)
        print(col2)
        raise Exception()

In [6]:
ragas_200_reference.to_csv(
    dataset_dir / "ragas_200_reference.csv", encoding="utf-8", index=False
)

In [7]:
# 모델별로 생성한 데이터를 읽어오고, ragas_200_ref와 일치하는지 검증
# 모든 모델의 에러 카운트가 0 이므로 질문이 일치함

for f in qa_data_files:
    df = pd.read_json(f)
    print(f)
    print(df.shape)
    error_count = 0
    for question in df["question"]:
        if question not in ragas_200_reference["user_input"].values:
            error_count += 1
    print("Error:", error_count)

datasets/generated_qa/result_kakaocorp.kanana_nano_2.json
(200, 2)
Error: 0
datasets/generated_qa/result_hyperclovax_seed_text_i.json
(200, 2)
Error: 0
datasets/generated_qa/result_exaone_3.5_2.4b_instruc.json
(200, 2)
Error: 0
datasets/generated_qa/result_google_gemma_3_4b_it_fr.json
(200, 2)
Error: 0
datasets/generated_qa/result_deepseek_deepseek_chat_.json
(200, 2)
Error: 0
datasets/generated_qa/result_gpt_4o_mini.json.json
(200, 2)
Error: 0
datasets/generated_qa/result_qwen_qwen3_4b_free.json.json
(198, 2)
Error: 0


##### 최종적으로 각 모델의 출력을 `ragas_200_with_model_answers.csv` 파일에 저장.

In [8]:
for f in qa_data_files:
    model_name = f.name.replace("result_", "").replace(".json", "")
    print(model_name)
    model_df = pd.read_json(f)
    for _, row in model_df.iterrows():
        question = row[0]  # 퀘스천 추출
        answer = row[1]
        mask = ragas_200_reference["user_input"] == question
        assert mask.sum() == 1

        if model_name not in ragas_200_reference.columns:
            ragas_200_reference[model_name] = None

        ragas_200_reference.loc[mask, model_name] = answer
ragas_200_reference.to_csv(
    dataset_dir / "ragas_200_with_model_answers.csv", index=False, encoding="utf-8"
)

kakaocorp.kanana_nano_2
hyperclovax_seed_text_i
exaone_3.5_2.4b_instruc
google_gemma_3_4b_it_fr
deepseek_deepseek_chat_
gpt_4o_mini
qwen_qwen3_4b_free


/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_80625/101684483.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  question = row[0]  # 퀘스천 추출
/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_80625/101684483.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[1]
/var/folders/lg/zz8kzx7j04q6lvkcyzwjznwh0000gn/T/ipykernel_80625/101684483.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

#### Ragas의 Evaluation에 맞게 데이터셋 생성

https://docs.ragas.io/en/stable/getstarted/rag_eval/#collect-evaluation-data

In [42]:
# TODO: item['reference_outputs]를 리스트 형태로 넣어야 됨
# 추후 RAGAS에서 EVAL 하기 위해서
from langchain_teddynote import logging

logging.langsmith("HUMETRO_EVAL_TEST")


import json

source_df = pd.read_csv(dataset_dir / "ragas_200_with_model_answers.csv")
eval_datasets = {}

for f in qa_data_files:
    dataset = []
    model_name = f.name.replace("result_", "").replace(".json", "")
    print("Create Eval Dataset for ", model_name)
    success_count = 0
    error_list = set()

    for item in source_df.to_dict("records"):
        debug_var = item["reference_outputs"]
        dataset.append(
            {
                "user_input": item["user_input"],
                "retrieved_contexts": [item["reference_outputs"]],
                "response": item[model_name],
                "reference": item["reference"],
            }
        )
    eval_datasets[model_name] = dataset

LangSmith 추적을 시작합니다.
[프로젝트명]
HUMETRO_EVAL_TEST
Create Eval Dataset for  kakaocorp.kanana_nano_2
Create Eval Dataset for  hyperclovax_seed_text_i
Create Eval Dataset for  exaone_3.5_2.4b_instruc
Create Eval Dataset for  google_gemma_3_4b_it_fr
Create Eval Dataset for  deepseek_deepseek_chat_
Create Eval Dataset for  gpt_4o_mini
Create Eval Dataset for  qwen_qwen3_4b_free


In [43]:
# 생성한 데이터셋 검증
eval_datasets.keys()
print([len(i) for i in eval_datasets.values()])

[200, 200, 200, 200, 200, 200, 200]


In [59]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas import EvaluationDataset

from langchain_openai import ChatOpenAI
from langchain_xai import ChatXAI
from langchain_openai import OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-4.1-2025-04-14")
llm = ChatXAI(model="grok-3-beta")
embeddings = OpenAIEmbeddings()
# testing llm response
response = llm.invoke("hello").content
print(response)

Hey there! How can I help you today? 😊


In [63]:
eval_datasets["qwen_qwen3_4b_free"]
eval_results

{'kakaocorp.kanana_nano_2': {'faithfulness': 0.4089, 'factual_correctness(mode=f1)': 0.3714},
 'hyperclovax_seed_text_i': {'faithfulness': 0.3661, 'factual_correctness(mode=f1)': 0.3020},
 'exaone_3.5_2.4b_instruc': {'faithfulness': 0.4023, 'factual_correctness(mode=f1)': 0.3710},
 'google_gemma_3_4b_it_fr': {'faithfulness': 0.5984, 'factual_correctness(mode=f1)': 0.4223},
 'deepseek_deepseek_chat_': {'faithfulness': 0.5682, 'factual_correctness(mode=f1)': 0.4400},
 'gpt_4o_mini': {'faithfulness': 0.6388, 'factual_correctness(mode=f1)': 0.4881},
 'qwen_qwen3_4b_free': None}

In [71]:
qwen_dataset = eval_datasets["qwen_qwen3_4b_free"]
fixed_qwen_dataset = []

for record in qwen_dataset:
    try:
        assert type(record["response"]) == str, print(record["response"])
        assert type(record["reference"]) == str
        assert type(record["retrieved_contexts"]) == list
        assert type(record["user_input"]) == str
        fixed_qwen_dataset.append(record)
    except AssertionError:
        continue
print(len(fixed_qwen_dataset))
eval_datasets["qwen_qwen3_4b_free"] = fixed_qwen_dataset

nan
nan
198


In [73]:
dataset = eval_datasets["qwen_qwen3_4b_free"]
print(len(dataset))

198


In [74]:
evaluator_llm = LangchainLLMWrapper(llm)
from ragas.metrics import Faithfulness, FactualCorrectness

eval_results = {}
# for key, dataset in eval_datasets.items():
dataset = eval_datasets["qwen_qwen3_4b_free"]
key = "qwen_qwen3_4b_free"
try:
    print(f"Evaluating {key}...")
    evaluation_dataset = EvaluationDataset.from_list(dataset)
    result = evaluate(
        dataset=evaluation_dataset,
        metrics=[Faithfulness(), FactualCorrectness()],
        llm=evaluator_llm,
    )
    print(key, "data_size", len(evaluation_dataset), " -> ", result)
    eval_results[key] = result
except Exception as e:
    print(f"Error evaluating {key}: {e}")
    eval_results[key] = None

with open("eval_results_200_grok.json", "w") as f:
    json.dump(eval_results, f)

Evaluating qwen_qwen3_4b_free...


Evaluating:   0%|          | 0/396 [00:00<?, ?it/s]

qwen_qwen3_4b_free data_size 198  ->  {'faithfulness': 0.5090, 'factual_correctness(mode=f1)': 0.4147}


TypeError: Object of type EvaluationResult is not JSON serializable

In [57]:
eval_result_for_four = {
    "kanana_nano_2": {"faithfulness": 0.3908, "factual_correctness(mode=f1)": 0.3561},
    "hyperclovax_seed_text_i": {
        "faithfulness": 0.3563,
        "factual_correctness(mode=f1)": 0.2941,
    },
    "Evaluating exaone_3.5_2.4b_instruc": {
        "faithfulness": 0.4134,
        "factual_correctness(mode=f1)": 0.3592,
    },
    "Evaluating google_gemma_3_4b_it_fr": {
        "faithfulness": 0.5712,
        "factual_correctness(mode=f1)": 0.4217,
    },
}

with open("eval_results_200_for_four.json", "w") as f:
    json.dump(eval_result_for_four, f)

In [75]:
eval_result_200_grok = {
    "kakaocorp.kanana_nano_2": {
        "faithfulness": 0.4089,
        "factual_correctness(mode=f1)": 0.3714,
    },
    "hyperclovax_seed_text_i": {
        "faithfulness": 0.3661,
        "factual_correctness(mode=f1)": 0.302,
    },
    "exaone_3.5_2.4b_instruc": {
        "faithfulness": 0.4023,
        "factual_correctness(mode=f1)": 0.371,
    },
    "google_gemma_3_4b_it_fr": {
        "faithfulness": 0.5984,
        "factual_correctness(mode=f1)": 0.4223,
    },
    "deepseek_deepseek_chat_": {
        "faithfulness": 0.5682,
        "factual_correctness(mode=f1)": 0.44,
    },
    "gpt_4o_mini": {"faithfulness": 0.6388, "factual_correctness(mode=f1)": 0.4881},
    "qwen_qwen3_4b_free": {
        "faithfulness": 0.509,
        "factual_correctness(mode=f1)": 0.4147,
    },
}
with open("eval_results_200_grok.json", "w") as f:
    json.dump(eval_result_200_grok, f)